# Task 2 - Model Building and Training

Objective: Build, train, and evaluate classification models to detect fraudulent transactions, using appropriate techniques for imbalanced data.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Add src to path
sys.path.append(os.path.abspath('../src'))
from modeling import prepare_data, get_preprocessor, train_and_evaluate, cross_validate_model

## 1. Data Preparation
### 1.1 Load Processed Datasets

In [ ]:
fraud_data = pd.read_csv('../data/processed/fraud_data_engineered.csv')
credit_data = pd.read_csv('../data/processed/creditcard_processed.csv')

print(f"Fraud Data Shape: {fraud_data.shape}")
print(f"Credit Card Data Shape: {credit_data.shape}")

### 1.2 Split Data (Stratified)

In [ ]:
# Fraud Data Preparation
fraud_drop = ['user_id', 'signup_time', 'purchase_time', 'device_id', 'ip_address']
X_fraud, y_fraud = prepare_data(fraud_data, 'class', fraud_drop)
X_f_train, X_f_test, y_f_train, y_f_test = train_test_split(X_fraud, y_fraud, test_size=0.2, stratify=y_fraud, random_state=42)

# Credit Data Preparation
X_credit, y_credit = prepare_data(credit_data, 'Class')
X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(X_credit, y_credit, test_size=0.2, stratify=y_credit, random_state=42)

print("Data split successfully.")

## 2. Baseline Model (Logistic Regression)
### 2.1 Fraud Data

In [ ]:
fraud_preprocessor = get_preprocessor(X_f_train)
lr_fraud = Pipeline(steps=[
    ('preprocessor', fraud_preprocessor),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

res_lr_fraud = train_and_evaluate(lr_fraud, X_f_train, X_f_test, y_f_train, y_f_test, "Logistic Regression (Fraud Data)")

### 2.2 Credit Card Data

In [ ]:
credit_preprocessor = get_preprocessor(X_c_train)
lr_credit = Pipeline(steps=[
    ('preprocessor', credit_preprocessor),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

res_lr_credit = train_and_evaluate(lr_credit, X_c_train, X_c_test, y_c_train, y_c_test, "Logistic Regression (Credit Data)")

## 3. Ensemble Model (Random Forest)
### 3.1 Fraud Data

In [ ]:
rf_fraud = Pipeline(steps=[
    ('preprocessor', fraud_preprocessor),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced'))
])

res_rf_fraud = train_and_evaluate(rf_fraud, X_f_train, X_f_test, y_f_train, y_f_test, "Random Forest (Fraud Data)")

### 3.2 Credit Card Data

In [ ]:
rf_credit = Pipeline(steps=[
    ('preprocessor', credit_preprocessor),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced'))
])

res_rf_credit = train_and_evaluate(rf_credit, X_c_train, X_c_test, y_c_train, y_c_test, "Random Forest (Credit Data)")

## 4. Cross-Validation

In [ ]:
# CV for Random Forest on Fraud Data
cv_rf_fraud = cross_validate_model(rf_fraud, X_fraud, y_fraud)

# CV for Random Forest on Credit Data
cv_rf_credit = cross_validate_model(rf_credit, X_credit, y_credit)

## 5. Model Comparison and Selection

In [ ]:
print("Summary of Results:")
print(f"Fraud Data - LR AUC-PR: {res_lr_fraud['auc_pr']:.4f}, RF AUC-PR: {res_rf_fraud['auc_pr']:.4f}")
print(f"Credit Data - LR AUC-PR: {res_lr_credit['auc_pr']:.4f}, RF AUC-PR: {res_rf_credit['auc_pr']:.4f}")

print("\nBased on the AUC-PR and F1 scores, the Random Forest model performs better on both datasets.")
print("The balanced class weights and ensemble nature handle the imbalance more effectively than baseline Logistic Regression.")